In [0]:
#################################### densnet model###############################################3
import os
from glob import glob
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.densenet import DenseNet201,DenseNet121
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import cv2
import matplotlib.pyplot as plt
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
import glob
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, Reshape, Activation,SimpleRNN,GRU, LSTM
from keras.layers.convolutional import Conv2D,Conv3D, Conv2DTranspose, Conv3DTranspose
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.wrappers import TimeDistributed
from sklearn.preprocessing import LabelBinarizer
import keras
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Dropout, Activation, Bidirectional, Conv2D, MaxPooling2D
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
#%Read the train & test Images and preprocessing
train_images = []
train_labels = [] 
for directory_path in glob.glob("/raid/Home/Users/aqayyum/pymultimodel/Classificationmodels/Datasetnew/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

label_to_id = {v:i for i,v in enumerate(np.unique(train_labels))}
id_to_label = {v: k for k, v in label_to_id.items()}
train_label_ids = np.array([label_to_id[x] for x in train_labels])
#image_data = image_data.reshape(-1,10,24,24,1)
train_images.shape, train_label_ids.shape, train_labels.shape
X_train1, y_train1, N_CATEGORY =train_images,train_label_ids,len(label_to_id)


ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1

# Produces test split.
x_remaining, x_test, y_remaining, y_test = train_test_split(X_train1, y_train1, test_size=ratio_test)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
x_train, x_val, y_train, y_val = train_test_split(x_remaining, y_remaining, test_size=ratio_val_adjusted)

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
from keras.utils.np_utils import to_categorical
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)
y_val=keras.utils.to_categorical(y_val, 3)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

X_train=x_train
X_test=x_test
X_val=x_val

#%matplotlib inline
import matplotlib.pyplot as plt

pre_trained_modeld = DenseNet121(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in pre_trained_modeld.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_modeld.layers))

#last_layer = pre_trained_modeld.get_layer('conv3_block12_concat')
#last_layer = pre_trained_modeld.get_layer('conv4_block24_concat')
#last_layer = pre_trained_modeld.get_layer('conv2_block6_concat')
last_layer = pre_trained_modeld.get_layer('relu')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu',name='dens_densnet')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(3, activation='softmax')(x)
# Configure and compile the model

modeldensnet201 = Model(pre_trained_modeld.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modeldensnet201.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

modeldensnet201.summary()

train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)
# Configure and compile the model

batch_size = 32
epochs = 3
history = modeldensnet201.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

modeldensnet201.summary()
for layer in pre_trained_modeld.layers:
    layer.trainable = True
    
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
modeldensnet201.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

modeldensnet201.summary()

batch_size = 64
epochs = 20
history = modeldensnet201.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

loss_val1, acc_val1 = modeldensnet201.evaluate(X_val, y_val, verbose=1)
#print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val1, loss_val1))

#np.save('validaccudense',acc_val1)
loss_test1, acc_test1 = modeldensnet201.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test1, loss_test1))
#np.save('testaccudense',acc_test1)
#
#model.save("DenseNetFullWA.h5")
#
###############################################33 Feature extraction from last layer#############################33
####################################Feature extraction from last layer of model##########################
layer_name = 'dens_densnet'
FC_layer_modeldense = Model(inputs=modeldensnet201.input,
                                 outputs=modeldensnet201.get_layer(layer_name).output)
print('last layer output shape:', FC_layer_modeldense.output_shape)
i=0
featuresd=np.zeros(shape=(train_images.shape[0],512))
#features=[]
for directory_path in glob.glob("/raid/Home/Users/aqayyum/pymultimodel/Classificationmodels/Datasetnew/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)    
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_modeldense.predict(img)
        featuresd[i]=FC_output
        i+=1
#Save the features of the train images to use it in future.
np.save('featuresdenseb5new', featuresd)
#
np.shape(featuresd)
##Name the feature rows as f_0, f_1, f_2...
feature_col=[]
for i in range(512):
    feature_col.append("f_"+str(i))
    i+=1
train_featuresd=pd.DataFrame(data=featuresd,columns=feature_col)
feature_col = np.array(feature_col)

train_class = list(np.unique(train_label_ids))
print('Training Features Shape:', train_featuresd.shape)
print('Training Labels Shape:', train_label_ids.shape)

X1=np.array(featuresd)
y=np.array(train_label_ids)
np.save('featuresdensepb6aug',X1)
np.save('labelsdatapb6aug',train_label_ids)
X_traind, X_testd, y_traind, y_testd = train_test_split(X1, y, test_size=0.2, stratify=y)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

parameters = {"n_neighbors": [1, 5, 10, 30],
              "weights": ['uniform', 'distance'],
              "metric": ['minkowski','euclidean','manhattan'],
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']}
kclf = KNeighborsClassifier()
kgclf = GridSearchCV(kclf, param_grid=parameters)
kgclf.fit(X_traind, y_traind)


kclf = kgclf.best_estimator_
kclf.fit(X_traind, y_traind)

y_testKNN = kclf.predict(X_testd)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#print_cmx(y_test1000.T[0], y_testKNN)
print(classification_report(y_testd, y_testKNN))
print("Accuracy: {0}".format(accuracy_score(y_testd, y_testKNN)))
modeldensnet201.save("DenseNetFullWagb6aug.h5")

In [0]:
# pretrained inception model used for feature extraction and also used other machine learning classifies for classification
import os
from glob import glob
#
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import cv2
import matplotlib.pyplot as plt
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
import glob
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, Reshape, Activation,SimpleRNN,GRU, LSTM
from keras.layers.convolutional import Conv2D,Conv3D, Conv2DTranspose, Conv3DTranspose
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.wrappers import TimeDistributed
from sklearn.preprocessing import LabelBinarizer
import keras
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Dropout, Activation, Bidirectional, Conv2D, MaxPooling2D
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


#%matplotlib inline
import matplotlib.pyplot as plt

import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.densenet import DenseNet201
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import cv2
import matplotlib.pyplot as plt
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
import glob
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, Reshape, Activation,SimpleRNN,GRU, LSTM
from keras.layers.convolutional import Conv2D,Conv3D, Conv2DTranspose, Conv3DTranspose
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.wrappers import TimeDistributed
from sklearn.preprocessing import LabelBinarizer
#%Read the train & test Images and preprocessing
train_images = []
train_labels = [] 
for directory_path in glob.glob("/raid/Home/Users/aqayyum/pymultimodel/Classificationmodels/dataset_wAG/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

label_to_id = {v:i for i,v in enumerate(np.unique(train_labels))}
id_to_label = {v: k for k, v in label_to_id.items()}
train_label_ids = np.array([label_to_id[x] for x in train_labels])
#image_data = image_data.reshape(-1,10,24,24,1)
train_images.shape, train_label_ids.shape, train_labels.shape

X_train1, y_train1, N_CATEGORY =train_images,train_label_ids,len(label_to_id)


ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1

# Produces test split.
x_remaining, x_test, y_remaining, y_test = train_test_split(X_train1, y_train1, test_size=ratio_test)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
x_train, x_val, y_train, y_val = train_test_split(x_remaining, y_remaining, test_size=ratio_val_adjusted)

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
from keras.utils.np_utils import to_categorical
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)
y_val=keras.utils.to_categorical(y_val, 3)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

X_train=x_train
X_test=x_test
X_val=x_val

#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X_train1, y_train1, test_size=0.20)
#X_train = X_train.astype('float32') / 255.
#X_test = X_test.astype('float32') / 255.
#from keras import utils as np_utils
##Y_train = np_utils.to_categorical(y_train, nb_classes)
#from keras.utils.np_utils import to_categorical
#y_train = keras.utils.to_categorical(Y_train, 3)
#y_test = keras.utils.to_categorical(Y_test, 3)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_train=np.array(X_train)
X_test=np.array(X_test)
X_val=np.array(X_val)
y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)
%matplotlib inline
import matplotlib.pyplot as plt

pre_trained_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))


last_layer = pre_trained_model.get_layer('mixed9')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu',name='dens_icept')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(3, activation='softmax')(x)

# Configure and compile the model

modelinception = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modelinception.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

modelinception.summary()
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')
print(X_train.shape)
train_datagen.fit(X_train)
print(X_val.shape)
val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

batch_size = 64
epochs = 3
history = modelinception.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))


for layer in pre_trained_model.layers:
    layer.trainable = True
    
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
modelinception.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)


modelinception.summary()

batch_size = 64
epochs = 20
history = modelinception.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])


loss_val, acc_val = modelinception.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
#np.save('validaccuincept',acc_val)
loss_test, acc_test = modelinception.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
#np.save('testaccuincept',acc_test)

###################################Feature extraction from last layer of model##########################
layer_name = 'dens_icept'
FC_layer_modelincep = Model(inputs=modelinception.input,
                                 outputs=modelinception.get_layer(layer_name).output)
print('last layer output shape:', FC_layer_modelincep.output_shape)
#%
i=0
features=np.zeros(shape=(train_images.shape[0],512))
#features=[]
for directory_path in glob.glob("/raid/Home/Users/aqayyum/pymultimodel/Classificationmodels/dataset_wAG/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)    
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_modelincep.predict(img)
        features[i]=FC_output
        i+=1
#Save the features of the train images to use it in future.
#np.save('featuresinceptionawg', features)
#
np.shape(features)
##Name the feature rows as f_0, f_1, f_2...
#feature_col=[]
#for i in range(512):
#    feature_col.append("f_"+str(i))
#    i+=1
#train_features=pd.DataFrame(data=features,columns=feature_col)
#feature_col = np.array(feature_col)
#
#train_class = list(np.unique(train_label_ids))
#print('Training Features Shape:', train_features.shape)
#print('Training Labels Shape:', train_label_ids.shape)

X=np.array(features)
y=np.array(train_label_ids)
np.save('featuresinceplast91',X)
np.save('labelsdatlast91',train_label_ids)
X_trainf, X_testf, y_trainf, y_testf = train_test_split(X, y, test_size=0.2, stratify=y)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

parameters = {"n_neighbors": [1, 5, 10, 30],
              "weights": ['uniform', 'distance'],
              "metric": ['minkowski','euclidean','manhattan'],
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']}
kclf = KNeighborsClassifier()
kgclf = GridSearchCV(kclf, param_grid=parameters)
kgclf.fit(X_trainf, y_trainf)


kclf = kgclf.best_estimator_
kclf.fit(X_trainf, y_trainf)

y_testKNN = kclf.predict(X_testf)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#print_cmx(y_test1000.T[0], y_testKNN)
print(classification_report(y_testf, y_testKNN))
print("Accuracy: {0}".format(accuracy_score(y_testf, y_testKNN)))
modelinception.save("InceptionV3last91.h5")
